# Let's train a LSTM to write Songs!

We're going to use [Keras](https://keras.io)  to write songs. At least 20 epochs are required before the generated text starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.


### Let's check out the corpus

In [1]:
from __future__ import print_function
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
import numpy as np
import pandas as pd
import random
import sys

# Read the entire file containing nietzsche's works
path = './data/songdata.csv'

df = pd.read_csv(path)
df.tail()

Using TensorFlow backend.


,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \nLet the angels fly l...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \nMore power \nPower to...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \nis something i'll believe \nf...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \nam i frightened \nwhere can ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \nmake yourself at home \ni'm a bit ...


In [2]:
df = df[df['artist']=='The Beatles']

In [15]:
text = df['text'].str.cat(sep='\n').lower()
# Output the length of the corpus
print('corpus length:', len(text))


# Create a sorted list of the characters
chars = sorted(list(set(text)))
print('total chars:', len(chars))

# Corpus is going to take tooooo long to train, so lets make it shorter
text = text[:1000000]
print('truncated corpus length:', len(text))

# Create a dictionary where given a character, you can look up the index and vice versa
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3

corpus length: 68056106
total chars: 50
truncated corpus length: 1000000


## Creates the overlapping windows with target characters

In [16]:

sentences = []
next_chars = []

# Step through the text via 3 characters at a time, taking a sequence of 40 bytes at a time. 
# There will be lots ofo overlap
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen]) # range from current index i for max length characters 
    next_chars.append(text[i + maxlen]) # the next character after that 
sentences = np.array(sentences)
next_chars = np.array(next_chars)
print('Number of sequences:', len(sentences))

Number of sequences: 333320


## Generates the 1 hot vectors for each character

In [5]:

def generator(sentences, next_chars, batch_size):
    X = np.zeros((batch_size, maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((batch_size, len(chars)), dtype=np.bool)
    length = len(sentences)
    index = 0
    while True:
        if index + batch_size >= length:
            index = 0
        X.fill(0)
        y.fill(0)
        for i in range(batch_size):
            sentence = sentences[index]
            for t, char in enumerate(sentence):
                X[i, t, char_indices[char]] = 1
            y[i, char_indices[next_chars[i]]] = 1
            index = index + 1
        yield X, y

        
def getdata(sentences, next_chars):
    X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
    length = len(sentences)
    index = 0
    for i in range(len(sentences)):
        sentence = sentences[i]
        for t, char in enumerate(sentence):
            X[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1
    return X, y

## Build the LSTM model

In [6]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

print("Compiling model complete...")


Build model...
Compiling model complete...


### Helper function to sample an index from a probability array
 The purpose of this function is to add some randomness so that the most likely character is not always chosen, and sometiems the 2nd or 3rd most likely cahracter is chosen

In [19]:

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

### And now the actual training...

In [7]:
diversity = 0.25
print('Diversity: ', diversity)

# Get data
#X, y = getdata(sentences, next_chars)

# The training
print('Training...')
batch_size = 128
#history = model.fit_generator(generator(sentences, next_chars, batch_size),steps_per_epoch=12800, epochs=10)

#history = model.fit(X, y,batch_size=128, epochs=30)


# Save the model
#model.save('songgenerator.h5')

Diversity:  0.25
Training...
Epoch 1/30
333320/333320 [==============================] - 92s - loss: 2.2098    
Epoch 2/30
333320/333320 [==============================] - 90s - loss: 1.8014    
Epoch 3/30
333320/333320 [==============================] - 89s - loss: 1.6613    
Epoch 4/30
333320/333320 [==============================] - 90s - loss: 1.5754    
Epoch 5/30
309248/333320 [==========================>...] - ETA: 6s - loss: 1.5160

KeyboardInterrupt: 

In [21]:
# Save the model
model = load_model('songgenerator.h5')

In [25]:
# Check out what our model predicts
sentence = 'behold, my field of cares\nalas, but there is nothing'
#sentence = 'why is my code note'

sentence = sentence[:maxlen]
x = np.zeros((1, maxlen, len(chars)))
for t, char in enumerate(sentence):
    x[0, t, char_indices[char]] = 1.
    
print(model.predict(x, verbose=0)[0])
print(sum(model.predict(x, verbose=0)[0]))

[  6.93858198e-08   3.79877136e-04   1.26710449e-06   4.52895529e-06
   1.69418563e-05   1.18955706e-10   8.03584186e-08   2.79000619e-06
   4.26955395e-08   2.13336170e-06   8.08551270e-09   1.56331809e-10
   8.13793824e-11   2.28181585e-09   1.01400677e-09   1.65661147e-15
   2.06823070e-09   1.28643402e-17   1.57628806e-11   2.76744643e-12
   2.64225131e-09   1.84367707e-06   3.36859880e-11   7.25217619e-09
   1.12565845e-04   1.18330512e-07   3.83770214e-07   4.08058696e-07
   9.97072816e-01   4.87218631e-06   8.58005421e-07   2.50542513e-07
   2.35300740e-05   1.74009500e-08   3.12264797e-06   2.76587542e-07
   2.26345219e-05   1.47651929e-06   2.28576385e-03   1.58739635e-06
   4.95234190e-07   3.98071534e-05   4.98848794e-06   1.75842695e-06
   2.32088905e-06   3.01764544e-06   4.19898561e-06   2.04144754e-08
   3.03970660e-06   3.72533506e-08]
0.999999964123


In [27]:
generated = ''
original = sentence
# Predict the next 400 characters based on the seed
for i in range(800):
    x = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x[0, t, char_indices[char]] = 1.

    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, 0.2)
    next_char = indices_char[next_index]

    generated += next_char
    sentence = sentence[1:] + next_char

print(original + generated)



some people still me a stars at the day  
  
i can see in your love  
  
i can't feel it wasn't move me  
you know i'm a migur  
do i real  
  
and i can't tell you now  
i forgot every word of the store  
  
come the night i don't care  
  
i can't tell you hold your mind  
i'm a migurs i could be  
i want to be want to see  
i want to be with you  
  
i want you move to tell you  
i'm a big down the time  
i can't tell me  
the story  
you want to be waiting out tonight  
don't turn the sun  
got a fine  
and i can't feel she love you  
i'm going to be  
i want to show i would be  
i want to be with you  
  
i can see the story  
when you take me to the time  
  
i'm going to be  
and i know the store  
don't think that i do  
i got the things i love you  
i'm going to be a girl  
  
i want to be want to see  
i want to be wa


is this the real life, is this just fantasy
easy come, easy the see you so  
i want to be your lonely and you see  
  
i say the love me do.  
  
i do see  
  
i say i'm not all the love is up  
i and i love you  
i'll be the love her better been  
round the done  
  
i see you know i want to her  
  
i say i'm not the love were  
  
i say i'm gonna can i got to be  
the sid me now  
and i'm not alone  
and i'm gonna cay  
i want to her  
  

In [24]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.